In [1]:
#dependencies and setup
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
from pprint import pprint
import os
import re
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\mosab\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [9]:
# Inspiration: https://www.blog.datahut.co/post/scraping-yahoo-finance-data-using-python
symbols = []
names = []
changes = []
percentChanges = []
prices = []
fifty_day_avgs = []
two_hundred_day_avgs = []
three_mnth_returns = []
ytd_returns = []
fifty_two_wks_rng = []

for i in range(0,8200+100,100):
    # visit the yahho finance URL
#     yahoo_finance_url = f"https://finance.yahoo.com/screener/9f0447a5-eb7c-4f0b-9936-fbdbc8909e8d?count=100&guccounter=1&guce_referrer=aHR0cHM6Ly9sb2dpbi55YWhvby5jb20v&guce_referrer_sig=AQAAAGPF95EkydeGJvNp6E9bsjTQPgE7tSGn3tp6JVHLpucCypyHJPcQKnR1Y5or9nnz5EIToPcQf4h-6rIe370xpWPvTGDUNQIcGjboPdXpx9zD7InHx464HAZtjqSlppReHVsIEdKCH-YNl7DgFswsFBvXHGZtugjqpFCSPiEi2TZ&offset={i}"
    yahoo_finance_url = f"https://finance.yahoo.com/screener/9656beca-4efe-45db-9286-50ed08a853fe?count=100&guccounter=1&guce_referrer=aHR0cHM6Ly9sb2dpbi55YWhvby5jb20v&guce_referrer_sig=AQAAAGPF95EkydeGJvNp6E9bsjTQPgE7tSGn3tp6JVHLpucCypyHJPcQKnR1Y5or9nnz5EIToPcQf4h-6rIe370xpWPvTGDUNQIcGjboPdXpx9zD7InHx464HAZtjqSlppReHVsIEdKCH-YNl7DgFswsFBvXHGZtugjqpFCSPiEi2TZ&offset={i}"
    browser.visit(yahoo_finance_url)
    time.sleep(4)

    # create HTML object
    html = browser.html

    # parse HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

#     response= requests.get(yahoo_finance_url)
#     soup=BeautifulSoup(response.text, 'html.parser')

    div = soup.find('div', class_ ='Ovx(a) Ovx(h)--print Ovy(h) W(100%)')
    table = div.find('table')
    for listing in table.find_all('tr', attrs={'class':'simpTblRow'}):
        for symbol in listing.find_all('td', attrs={'aria-label':'Symbol'}):
            symbols.append(symbol.text)
        for name in listing.find_all('td', attrs={'aria-label':'Name'}):
            names.append(name.text)
        for change in listing.find_all('td', attrs={'aria-label':'Change'}):
            changes.append(change.text)
        for percentChange in listing.find_all('td', attrs={'aria-label':'% Change'}):
            percentChanges.append(percentChange.text)
        for price in listing.find_all('td', attrs={'aria-label':'Price (Intraday)'}):
            prices.append(price.text)
        for fifty_day_avg in listing.find_all('td', attrs={'aria-label':'50 Day Average'}):
            fifty_day_avgs.append(fifty_day_avg.text)
        for two_hundred_day_avg in listing.find_all('td', attrs={'aria-label':'200 Day Average'}):
            two_hundred_day_avgs.append(two_hundred_day_avg.text)
        three_mnth_returns.append(listing.select('td')[7].text)
        ytd_returns.append(listing.select('td')[8].text)
        for ranges in listing.find_all('td', attrs={'aria-label':'52 Week Range'}):
            fifty_two_wks_rng.append(ranges.text)

In [10]:
# close the browser session    
browser.quit()

In [11]:
df = pd.DataFrame({"Symbol": symbols,"names": names, "changes": changes, "percentChanges": percentChanges,\
                  "prices": prices,"fifty_day_avgs":fifty_day_avgs, "two_hundred_day_avgs":two_hundred_day_avgs, \
                   "three_mnth_returns":three_mnth_returns, "ytd_returns":ytd_returns})
df.head()

,Symbol,names,changes,percentChanges,prices,fifty_day_avgs,two_hundred_day_avgs,three_mnth_returns,ytd_returns
0,LSHUX,Kinetics Spin-Off and Corporate Restructuring ...,+1.15,+6.25%,18.40,18.45,19.80,-7.63%,42.97%
1,LSHCX,Kinetics Spin-Off and Corporate Restructuring ...,+1.06,+6.24%,16.98,17.04,18.32,-7.92%,41.74%
2,LSHAX,Kinetics Spin-Off and Corporate Restructuring ...,+1.13,+6.18%,18.29,18.34,19.69,-7.67%,42.67%
3,WWNPX,Kinetics Paradigm Fund No Load Class,+4.55,+5.96%,76.36,74.87,78.55,-2.74%,41.43%
4,KNPCX,Kinetics Paradigm Fund Advisor Class C,+4.00,+5.96%,67.13,65.85,69.19,-2.92%,40.53%


In [12]:
df.info()
# df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8200 entries, 0 to 8199
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Symbol                8200 non-null   object
 1   names                 8200 non-null   object
 2   changes               8200 non-null   object
 3   percentChanges        8200 non-null   object
 4   prices                8200 non-null   object
 5   fifty_day_avgs        8200 non-null   object
 6   two_hundred_day_avgs  8200 non-null   object
 7   three_mnth_returns    8200 non-null   object
 8   ytd_returns           8200 non-null   object
dtypes: object(9)
memory usage: 576.7+ KB


In [13]:
df.columns

Index(['Symbol', 'names', 'changes', 'percentChanges', 'prices',
       'fifty_day_avgs', 'two_hundred_day_avgs', 'three_mnth_returns',
       'ytd_returns'],
      dtype='object')

In [14]:
for col in df.columns[2:]:
    df[f"{col}"] = df[f"{col}"].str.replace('%', '')
    df[f"{col}"] = df[f"{col}"].str.replace(',', '')
    df[f"{col}"] = pd.to_numeric(df[f"{col}"], errors='coerce')

In [15]:
df.head()

,Symbol,names,changes,percentChanges,prices,fifty_day_avgs,two_hundred_day_avgs,three_mnth_returns,ytd_returns
0,LSHUX,Kinetics Spin-Off and Corporate Restructuring ...,1.15,6.25,18.40,18.45,19.80,-7.63,42.97
1,LSHCX,Kinetics Spin-Off and Corporate Restructuring ...,1.06,6.24,16.98,17.04,18.32,-7.92,41.74
2,LSHAX,Kinetics Spin-Off and Corporate Restructuring ...,1.13,6.18,18.29,18.34,19.69,-7.67,42.67
3,WWNPX,Kinetics Paradigm Fund No Load Class,4.55,5.96,76.36,74.87,78.55,-2.74,41.43
4,KNPCX,Kinetics Paradigm Fund Advisor Class C,4.00,5.96,67.13,65.85,69.19,-2.92,40.53


In [16]:
df.info()
# df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8200 entries, 0 to 8199
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Symbol                8200 non-null   object 
 1   names                 8200 non-null   object 
 2   changes               8200 non-null   float64
 3   percentChanges        8200 non-null   float64
 4   prices                8200 non-null   float64
 5   fifty_day_avgs        8200 non-null   float64
 6   two_hundred_day_avgs  8200 non-null   float64
 7   three_mnth_returns    8173 non-null   float64
 8   ytd_returns           8171 non-null   float64
dtypes: float64(7), object(2)
memory usage: 576.7+ KB


In [17]:
df.describe()

,changes,percentChanges,prices,fifty_day_avgs,two_hundred_day_avgs,three_mnth_returns,ytd_returns
count,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000,8173.000000,8171.000000
mean,0.185134,0.472424,40.099615,38.622411,38.083830,4.455532,18.456491
std,0.464000,0.648519,50.546713,48.088657,46.912244,4.028798,10.288467
min,-5.040000,-3.060000,14.660000,13.990000,6.570000,-16.040000,-25.830000
25%,0.040000,0.150000,19.790000,19.280000,19.160000,2.180000,12.470000
50%,0.100000,0.370000,27.405000,26.545000,26.315000,4.340000,18.980000
75%,0.240000,0.680000,44.492500,42.860000,42.280000,6.600000,24.950000
max,9.720000,6.250000,2082.940000,1989.200000,1943.890000,35.650000,91.560000


In [18]:
df.to_csv('yahoo_finance_mutual_fund.csv', index = False)